In [ ]:
from io import StringIO
import requests
import pandas as pd
from bs4 import BeautifulSoup
from  datetime import datetime , timedelta
import bs4
import numpy as np

In [18]:
tick = "IDEA"

In [19]:
url = "https://www.screener.in/company/"+tick+"/consolidated/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers)


In [20]:
soup = BeautifulSoup(response.content, 'html.parser')
section = soup.find('section', id='profit-loss')
table = section.find('table', class_='data-table responsive-text-nowrap')
thead = table.find('thead')
tbody = table.find('tbody')

headers = [header.get_text(strip=True) for header in thead.find_all('th')]

data_rows = []
for row in tbody.find_all('tr'):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(['td', 'th'])]
    data_rows.append(row_data)

df = pd.DataFrame(data_rows, columns=headers)

In [21]:
df

,,Mar 2012,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,TTM
0,Sales+,"19,489","22,407","26,519","31,571","35,949","35,576","28,279","37,092","44,958","41,952","38,516","42,177","42,577"
1,Expenses+,"14,440","16,439","18,237","20,771","24,281","25,348","22,224","32,976","30,042","25,006","22,547","25,424","25,579"
2,Operating Profit,"5,048","5,969","8,282","10,800","11,668","10,227","6,054","4,116","14,916","16,946","15,968","16,753","16,998"
3,OPM %,26%,27%,31%,34%,32%,29%,21%,11%,33%,40%,41%,40%,40%
4,Other Income+,44,131,247,497,641,746,703,"1,789","-36,964","-19,563",363,354,908
5,Interest,"1,056","1,045",966,"1,060","1,803","4,010","4,847","9,545","15,393","17,998","20,981","23,354","24,487"
6,Depreciation,"2,981","3,478","4,519","5,304","6,256","7,827","8,409","14,536","24,356","23,638","23,584","23,050","22,586"
7,Profit before tax,"1,055","1,577","3,044","4,933","4,250",-863,"-6,499","-18,175","-61,797","-44,253","-28,234","-29,298","-29,168"
8,Tax %,31%,36%,35%,35%,36%,54%,36%,20%,-20%,0%,-0%,-0%,
9,Net Profit+,723,"1,011","1,968","3,193","2,728",-400,"-4,168","-14,604","-73,878","-44,233","-28,245","-29,301","-29,983"


In [22]:
dates = list(df.columns)[1:len(df.columns)-1]
dates = [['1-03-' +date.split(' ')[1], '10-03-' +date.split(' ')[1] ] for date in dates]

In [23]:
session = requests.session()
headers = {
    "user-agent": "Chrome/87.0.4280.88"
}
head = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/87.0.4280.88 Safari/537.36 "
}


def getHistoryData(company, from_date=(datetime.today().strftime("%d-%m-%Y")), to_date=(datetime(datetime.today().year - 1, datetime.today().month,datetime.today().day).strftime("%d-%m-%Y"))):
    session.get("https://www.nseindia.com", headers=head)
    session.get("https://www.nseindia.com/get-quotes/equity?symbol=" + company, headers=head) 
    session.get("https://www.nseindia.com/api/historical/cm/equity?symbol="+company, headers=head)
    url = "https://www.nseindia.com/api/historical/cm/equity?symbol=" + company + "&series=[%22EQ%22]&from=" + from_date + "&to=" + to_date + "&csv=true"
    webdata = session.get(url=url, headers=head)
    df = pd.read_csv(StringIO(webdata.text[3:]))
    return df



price = []
for i in dates:
    x = getHistoryData(tick,from_date=i[0],to_date=i[1])['OPEN '].iloc[0]
    x = str(x)
    x = x.replace(",", "")
    x = x.split('.')[0]
    price.append(int(x))

In [24]:
# Historical price data 
print(price)

[94, 113, 134, 149, 107, 104, 80, 32, 3, 10, 10, 6]


In [25]:
temp = list(df.loc[df[''] == 'EPS in Rs'].iloc[0])[1:]
eps = []
for i in temp:
    eps.append(int(i.split(".")[0]) + (int(i.split(".")[1])/(10**len(i.split(".")[1]))))

ttm = eps[-1]
eps = eps[:-1]
pe = np.array(price)/np.array(eps)

In [26]:
print(eps)

[0.66, 0.93, 1.8, 2.7, 2.3, 0.34, -1.0899999999999999, -4.92, -24.29, -14.61, -7.21, -5.98]


In [27]:
mid_point = len(eps) // 2
eps_mean_first_half = np.mean(eps[:mid_point])
eps_mean_second_half = np.mean(eps[mid_point:])

if eps_mean_second_half > eps_mean_first_half:
    print("Stable Earning Pattern")
else :
    print("Unstable Earning Pattern")

Unstable Earning Pattern


In [28]:
print(pe)

[ 1.42424242e+02  1.21505376e+02  7.44444444e+01  5.51851852e+01
  4.65217391e+01  3.05882353e+02 -7.33944954e+01 -6.50406504e+00
 -1.23507616e-01 -6.84462697e-01 -1.38696255e+00 -1.00334448e+00]


In [29]:
# EPS Growth Rate
eps_gr = (eps[-1]/eps[0])**(1/len(eps))*100
print(eps_gr)

(116.06649831807694+31.099924492636976j)


In [30]:
# Intrinsic P/E or Average P/E
pe_avg = np.average(pe)
print(pe_avg)

55.23887522244769


In [31]:
# P/E Growth Rate
pe_gr = (pe[-1]/pe[0])**(1/len(pe))*100
print(pe_gr)

nan


/var/folders/yl/5879yj014q11kbqcnjh_rd0m0000gn/T/ipykernel_1001/3082972927.py:2: RuntimeWarning: invalid value encountered in double_scalars
  pe_gr = (pe[-1]/pe[0])**(1/len(pe))*100


In [32]:
# Intrinsic Value
pe_avg * ttm

-322.5950312990945

In [1]:
import pickle

# Load the pickled list from a file
with open('/Users/satyam/Desktop/Projects/ACM/Final/GPT-Fund/news_.pkl', 'rb') as f:
    loaded_list = pickle.load(f)


[" Summary: ABB India reported strong financial performance in Q3, with revenue growth moderating but broad-based strong performance continuing. The company has seen growth in new segments such as renewables and data centers. However, the ongoing global economic uncertainty and geopolitical tensions pose risks to the company's growth prospects.\n\nRisk Measure: Moderate. The risk level is moderate due to the ongoing global economic uncertainty and geopolitical tensions, which could impact the company's growth prospects. Additionally, the company's exposure to currency fluctuations and commodity price volatility could also impact its financial performance.\n\nRecommendation: Hold. Given the moderate risk level and the company's strong financial performance, it's advisable for investors to hold their positions on ABB India. However, investors should closely monitor the company's performance and the global economic and geopolitical environment to make informed investment decisions.\n\nRat

In [3]:
print(loaded_list[0])

 Summary: ABB India reported strong financial performance in Q3, with revenue growth moderating but broad-based strong performance continuing. The company has seen growth in new segments such as renewables and data centers. However, the ongoing global economic uncertainty and geopolitical tensions pose risks to the company's growth prospects.

Risk Measure: Moderate. The risk level is moderate due to the ongoing global economic uncertainty and geopolitical tensions, which could impact the company's growth prospects. Additionally, the company's exposure to currency fluctuations and commodity price volatility could also impact its financial performance.

Recommendation: Hold. Given the moderate risk level and the company's strong financial performance, it's advisable for investors to hold their positions on ABB India. However, investors should closely monitor the company's performance and the global economic and geopolitical environment to make informed investment decisions.

Rating: 7/1

In [4]:
import pandas as pd
df = pd.read_csv("/Users/satyam/Desktop/Projects/ACM/Final/GPT-Fund/stocks_saved.csv")

In [5]:
df

,Unnamed: 0.1,Unnamed: 0,Stock,Intrinsic_Value,Price,Risk,Recomm,Score,Value
0,0,0,ABB,Unstable,5841.35,1,1,7,-1.000000
1,1,1,ADAI,601.8401498820529,1063.75,1,1,7,0.565772
2,2,2,ADEL,Unstable,3207.35,0,2,9,-1.000000
3,3,3,ADNA,4561.2249818719265,1930.30,1,1,7,2.362962
4,4,4,APSE,768.9661002074058,1326.95,0,2,9,0.579499
...,...,...,...,...,...,...,...,...,...
83,83,83,UNBK,Unstable,153.75,0,2,8,-1.000000
84,84,84,VARB,3344.00146566806,1414.25,1,2,7,2.364505
85,85,85,VDAN,68.52418418542165,275.35,1,1,7,0.248862
86,86,86,WIPR,687.370950841566,514.55,2,0,4,1.335868
